<a href="https://colab.research.google.com/github/eddorileo/saudeDigital/blob/main/src/AnaliseExploratoriaDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ciência de Dados e Saúde Digital
##Estudo de Caso: Absenteísmo (No-show)
Facilitador: Éderson Dorileo, MSc, PhD


###1. Importação e Limpeza dos Dados

In [2]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar o estilo dos gráficos
sns.set(style="whitegrid")

# Carregar a base de dados
df = pd.read_csv('no_show.csv')

# Visualizar as primeiras linhas da base de dados
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [6]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


###2. Tratamento de Dados Nulos e Inconsistências

In [3]:
#deletar colunas desnecessarias
df = df.drop(columns=['PatientId', 'AppointmentID'])

# Verificar valores nulos
#df.isnull().sum()

# Tratar valores nulos (se necessário)
df = df.dropna()

# Exemplo: Verificar idade negativa
df = df[df['Age'] >= 0]

# Verificar duplicatas
df = df.drop_duplicates()

# Transformar a variável categórica 'No-show' em numérica
# Dados categóricos não são adequados para correlação Pearson
# Transformação está ok para correlação Spearman (ordinais ou binarios)
df['No-show'] = df['No-show'].map({'Yes': 1, 'No': 0}).copy()

# Transformar a variável categórica 'Gender' em numérica
df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})

# Filtrar colunas numéricas, incluindo a variável target transformada
numerical_cols = df.select_dtypes(include=[np.number]).columns

# Remover valores nulos (se houver) nas colunas numéricas
df_numerical = df[numerical_cols].dropna()

###3. Elementos da tabela
Título:
Número de casos de diabetes nos 5 bairros com maiores incidência

In [11]:

# Convertendo 'AppointmentDay' para datetime e extraindo o dia da semana
# df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
# df['DayOfWeek'] = df['AppointmentDay'].dt.day_name(locale='pt_BR.UTF-8')
df['DayOfWeek'] = df['AppointmentDay'].dt.day_name()


# Mapeando os dias da semana para português (seg, ter, qua, qui, sex)
day_mapping = {
    "Monday": "seg",
    "Tuesday": "ter",
    "Wednesday": "qua",
    "Thursday": "qui",
    "Friday": "sex",
    "Saturday": "sáb",
    "Sunday": "dom"
}
df['DayOfWeek'] = df['DayOfWeek'].map(day_mapping)

# Organizando a tabela para contabilizar "No-show" por dia da semana
table = df.groupby(['DayOfWeek', 'No-show']).size().unstack(fill_value=0)

# Ordenando a tabela pelos dias da semana em ordem crescente
day_order = ["seg", "ter", "qua", "qui", "sex", "sáb", "dom"]
table = table.reindex(day_order)

# Exibindo a tabela
print(table)

No-show         No     Yes
DayOfWeek                 
seg        18025.0  4690.0
ter        20488.0  5152.0
qua        20774.0  5093.0
qui        13909.0  3338.0
sex        14982.0  4037.0
sáb           30.0     9.0
dom            NaN     NaN


In [7]:

# Criando a tabela de contingência para "Gender" e "No-show"
contingency_table = pd.crosstab(df['Gender'], df['No-show'])

# Renomeando os rótulos das linhas e colunas
contingency_table.index = ["Mulher", "Homem"]
contingency_table.columns = ["Não", "Sim"]

# Adicionando linha e coluna "Total"
contingency_table["Total"] = contingency_table.sum(axis=1)
total_row = pd.DataFrame(contingency_table.sum(axis=0)).T
total_row.index = ["Total"]
contingency_table = pd.concat([contingency_table, total_row])

# Exibindo a tabela de contingência
print(contingency_table)


          Não    Sim   Total
Mulher  56961  14455   71416
Homem   30832   7643   38475
Total   87793  22098  109891
